In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import unicodedata
import re
import io

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive');

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path_to_file = '/content/drive/MyDrive/Colab Notebooks/GeekBrains/NLP/Project/comedy.txt'

In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='1251')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 808629 characters


In [5]:
print(text[:500])

 * АД * 


ПЕСНЬ ПЕРВАЯ


                    1 Земную жизнь пройдя до половины,
                      Я очутился в сумрачном лесу,
                      Утратив правый путь во тьме долины.

                    4 Каков он был, о, как произнесу,
                      Тот дикий лес, дремучий и грозящий,
                      Чей давний ужас в памяти несу!

                    7 Так горек он, что смерть едва ль не слаще.
                      Но, благо в нем обретши навсегда,
      


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique words'.format(len(vocab)))

130 unique words


In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [12]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 512

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [14]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [15]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 130) # (batch_size, sequence_length, vocab_size)


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           33280     
_________________________________________________________________
gru (GRU)                    (64, None, 512)           1182720   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 512)           1575936   
_________________________________________________________________
gru_2 (GRU)                  (64, None, 512)           1575936   
_________________________________________________________________
dense (Dense)                (64, None, 130)           66690     
Total params: 4,434,562
Trainable params: 4,434,562
Non-trainable params: 0
_________________________________________________________________


In [17]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [18]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [19]:
model.compile(optimizer='adam', loss=loss)

In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = './checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=800,
    save_weights_only=True)

In [21]:
EPOCHS = 500

In [22]:
history = model.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback, early_stopping])

Epoch 1/500
125/125 [==============================] - 12s 70ms/step - loss: 2.6453
Epoch 2/500
125/125 [==============================] - 9s 70ms/step - loss: 1.6347
Epoch 3/500
125/125 [==============================] - 10s 71ms/step - loss: 1.4469
Epoch 4/500
125/125 [==============================] - 10s 71ms/step - loss: 1.3203
Epoch 5/500
125/125 [==============================] - 10s 71ms/step - loss: 1.2314
Epoch 6/500
125/125 [==============================] - 10s 71ms/step - loss: 1.1738
Epoch 7/500
125/125 [==============================] - 10s 72ms/step - loss: 1.1311
Epoch 8/500
125/125 [==============================] - 10s 72ms/step - loss: 1.0941
Epoch 9/500
125/125 [==============================] - 10s 72ms/step - loss: 1.0613
Epoch 10/500
125/125 [==============================] - 10s 72ms/step - loss: 1.0347
Epoch 11/500
125/125 [==============================] - 10s 72ms/step - loss: 1.0137
Epoch 12/500
125/125 [==============================] - 10s 73ms/step - los